## Creating a VR Trajectory Viewer with Narupa and MDAnalysis


In this notebook, we set up our own VR trajectory viewer from scratch using MDAnalysis and Narupa, adding playback controls that can be controlled from VR.

## Reading in the trjaectory 

We've got a small trajectory of oseltamivir unbinding from neuraminidase, produced with iMD-VR. 

In [ ]:
import MDAnalysis as mda

In [ ]:
topology_file = 'files/3TI6_ose_wt.pdb'
trajectory_file = 'files/ose_wt.dcd'

In [ ]:
universe = mda.Universe(topology_file, trajectory_file, trajectory=True, guess_bonds=True)

In [ ]:
n_frames = universe.trajectory.n_frames
n_frames

## Setting up the Trajectory Server

Now we've got the trajectory, let's serve it with Narupa. We'll create the server, and then send frames one at a time

In [ ]:
from narupa.app import NarupaFrameApplication
from narupa.mdanalysis import mdanalysis_to_frame_data

In [ ]:
frame_server = NarupaFrameApplication.basic_server(port=0)

In [ ]:
current_frame_index = 0

We just need to send the topology once, then the positions after that, so we define a couple of methods 
to help

In [ ]:
def send_topology_frame():
    # Convert the mdanalysis topology to a Narupa frame
    frame = mdanalysis_to_frame_data(universe, topology=True, positions=False)
    frame_server.frame_publisher.send_frame(0, frame)

In [ ]:
def send_frame(index):
    # Send the particle positions of the given trajectory index. 
    assert 0 <= index < universe.trajectory.n_frames, f'Frame index not in range [{0},{universe.trajectory.n_frames-1}]'
    time_step = universe.trajectory[index]
    frame = mdanalysis_to_frame_data(universe, topology=False, positions=True)
    frame_server.frame_publisher.send_frame(index, frame)

In [ ]:
send_topology_frame()

In [ ]:
send_frame(5)

If you send frames one at a time like this, you should see something like this in VR: 

![Manual Trajectory](./images/narupa_neuraminidase_trajectory_manual.gif)

## Make it loop on it's own

This is cool, but it's a bit annoying having to send frames manually, let's instead make it play by itself. 

In [ ]:
playback_fps = 15 # Frames per second

In [ ]:
playback_seconds = 10 # How long to play for, in seconds

In [ ]:
from datetime import datetime, timedelta
import time

The following cell sends `playback_fps`  frames a second for `playback_seconds`

In [ ]:
start_time = datetime.now()
index = 0
playback_time = timedelta(seconds=playback_seconds) # Represents time difference in seconds
while datetime.now() - start_time < playback_time:
    send_frame(index)
    index = (index + 1) % universe.trajectory.n_frames
    time.sleep( 1 / playback_fps) # Delay sending frames so we hit the desired FPS

This results in pretty smooth looping playback:

![Narupa looping trajectory](./images/narupa_neuraminidase_trajectory_loop.gif)

## Making it even better

We've now got something that can 'play' trajectories, but it would be good if we could leave it running, infinitely looping, 
and not block execution of other cells. It would also be nice to be able to pause, reset and step forward the trajectory. 

The following cell creates a small class for doing this. Given our `universe` and `frame_server` that we defined above, it sets up some logic for running playback on a background thread.

**Note**: Creating classes and running background threads is quite complicated for a Jupyter Notebook, and is only intended as a learning example! If you decide to customize this further, it would be a good idea to move these ideas to their own python modules.

In [ ]:
from threading import RLock
from concurrent import futures

class TrajectoryPlayback:
    
    def __init__(self):        
        """
        Initialise playback, setting things up.
        """
        # Get a pool of threads (just one) that we can run the play back on
        self.threads = futures.ThreadPoolExecutor(max_workers=1)
        self._run_task = None
        self._cancelled = False
        self._cancel_lock = RLock()
        self.frame_index = 0
    
    @property 
    def is_running(self):
        # Fancy logic that just checks whether or not we're playing the trajectory in the background
        return self._run_task is not None and not (self._run_task.cancelled() or self._run_task.done())

    def play(self):
        """
        Plays the trajectory in the background.
        """
        # First, we have to cancel any existing playback, and start a new one.
        with self._cancel_lock:
            self.cancel_playback(wait=True)
        self.run_playback()
        
    def step(self):
        """
        Take a single step of the trajectory and stop. 
        """
        # The lock here ensures only one person can cancel at a time. 
        with self._cancel_lock:
            self.cancel_playback(wait=True)
            self._step_one_frame()

    def pause(self):
        """
        Pause the playback, by cancelling any current playback.
        """
        with self._cancel_lock:
            self.cancel_playback(wait=True)

    def run_playback(self, block=False):
        """
        Runs the trajectory playback. If block is False, it will run on a background thread.
        """
        if self.is_running:
            raise RuntimeError("The trajectory is already playing on a thread!")
        if block:
            self._run()
        else:
            self._run_task = self.threads.submit(self._run)
    
    def _run(self):
        while not self._cancelled:
            self._step_one_frame()
            time.sleep( 1 / playback_fps) # Delay sending frames so we hit the desired FPS
        self._cancelled = False
        
    def _step_one_frame(self):
        send_frame(self.frame_index)
        self.frame_index = (self.frame_index + 1) % universe.trajectory.n_frames

    def cancel_playback(self, wait=False):
        """
        Cancel trajectory playback, if it's running. If wait is True, this method will wait until the playback stops 
        before returning.
        """
        if self._run_task is None:
            return

        if self._cancelled:
            return
        self._cancelled = True
        if wait:
            self._run_task.result()
            self._cancelled = False

    def reset(self):
        self.frame_index = 0


We've defined our class, now we have to instantiate it: 

In [ ]:
trajectory_player = TrajectoryPlayback()

Try running these cells and see what happens in VR

In [ ]:
trajectory_player.play()

In [ ]:
trajectory_player.cancel_playback()

In [ ]:
trajectory_player.play()

In [ ]:
trajectory_player.pause()

In [ ]:
trajectory_player.play()

In [ ]:
trajectory_player.reset()

In [ ]:
trajectory_player.step()

In [ ]:
trajectory_player.step()

## Control it from VR 

This is nearly there! Now let's just wire up some commands, so the VR buttons for pause, play and step will control our server. Since these are standard commands used by other applications, we store their names: 

In [ ]:
from narupa.trajectory.frame_server import PLAY_COMMAND_KEY, RESET_COMMAND_KEY, STEP_COMMAND_KEY, PAUSE_COMMAND_KEY

In [ ]:
print(PLAY_COMMAND_KEY, RESET_COMMAND_KEY, STEP_COMMAND_KEY, PAUSE_COMMAND_KEY)

Now we tell the server that we want our `play`, `pause`, `step` and `reset` methods to be called whenever those commands are run by a client.

In [ ]:
frame_server.server.register_command(PLAY_COMMAND_KEY, trajectory_player.play)
frame_server.server.register_command(PAUSE_COMMAND_KEY, trajectory_player.pause)
frame_server.server.register_command(RESET_COMMAND_KEY, trajectory_player.reset)
frame_server.server.register_command(STEP_COMMAND_KEY, trajectory_player.step)

Now it can be controlled from the iMD-VR app!

## Making it Pretty 

Let's change how it looks. We've covered this in more detail in the [neuraminidase iMD](../ase/openmm_neuraminidase.ipynb) and [graphene](../ase/openmm_graphene.ipynb) examples.

**Note**: A lot of this will probably be made simpler in upcoming releases

In [ ]:
from narupa.app import NarupaImdClient
client = NarupaImdClient.connect_to_single_server(port=frame_server.port)
client.subscribe_to_frames()
client.wait_until_first_frame()

In [ ]:
client.subscribe_multiplayer()

In [ ]:
import matplotlib.cm

def get_matplotlib_gradient(name: str):
    cmap = matplotlib.cm.get_cmap(name)
    return list(list(cmap(x/7)) for x in range(0, 8, 1))

In [ ]:
from narupa.mdanalysis import frame_data_to_mdanalysis
def generate_mdanalysis_selection(selection: str):
    universe = frame_data_to_mdanalysis(client.first_frame)
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

In [ ]:
root_selection = client.root_selection
with root_selection.modify():
    root_selection.hide = True
    root_selection.interaction_method = 'none'

In [ ]:
protein = client.create_selection("Protein", [])

In [ ]:
with protein.modify():
    protein.set_particles(generate_mdanalysis_selection("protein and not type H"))

In [ ]:
with protein.modify():
    protein.renderer = {
            'sequence': 'polypeptide',
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('viridis')
            },
            'render': 'geometric spline',
            'scale': 0.2
        }

In [ ]:
# Select ligand
ligand = client.create_selection("Ligand", [])
with ligand.modify():
    ligand.set_particles(generate_mdanalysis_selection("resname OSE"))

In [ ]:
with ligand.modify():
    ligand.renderer = {
            'scale': 0.1,
            'render': 'liquorice',
            'color': 'cpk',
        }

## Tidying Up

In [ ]:
trajectory_player.cancel_playback()
client.close()
frame_server.close()

# Next Steps

* Understand how [frames](../fundamentals/frame.ipynb) are constructed. 
* See a detailed example of setting up custom [commands](../fundamentals/commands_and_state.ipynb).